In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import requests
from selenium import webdriver
import time 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from urllib.parse import urljoin
import csv

In [7]:
def get_profile_urls(list_of_schools):   
    profile_urls = []
    for school in list_of_schools:
        print(f"Going through {school}".format(school))
        print(len(profile_urls))
        driver = webdriver.Chrome()
        driver.get("https://www.admitsee.com/searchundergrad")
        
        time.sleep(1)
        
        try:
            #Search
            inputElement = driver.find_element_by_id("combined_search1")
            inputElement.send_keys(school)

            time.sleep(3)

            #click on top search result 
            driver.find_element_by_xpath("//li[@class='result']/a").click()

            #click on Go 
            driver.find_element_by_xpath("//input[@value='GO']").click()
            
        except Exception as e:
            print(f"error in searching for...{school}".format(school))

    
        i = 1
        screen_height = driver.execute_script("return window.screen.height;")
        #scroll down
        while True:
            try:
                driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))

                i+=1

                time.sleep(3)

                scroll_height = driver.execute_script("return document.body.scrollHeight;")

                if (screen_height * i) > scroll_height:
                    temp = driver.page_source
                    break      

            except Exception as e:
                print(e)
                temp = driver.page_source
                break

        #after breaking read profile link and store
        try: 
            links = driver.find_elements_by_xpath("//div[@class='profile_card_button']/a")
            profile_urls +=[link.get_attribute("href") for link in links] 
        except Exceptionception as e: 
            print(f"problem retrieving links")
            
        driver.close()
        
    return profile_urls

        
    

In [49]:
def get_user_info(list_of_user_url):
    user_dictionary = {}
    
    with open('example_datafile.csv', 'w', newline='') as csvfile:
        fieldnames = ["user_url","username","current_school","user_major","hometown",
                     "gender","ethnicity","English_FL","Admission", "class_of","first_gen",
                     "accepted_schools","waitlisted_schools","rejected_schools","legacy","GPA_measure",
                     "GPA","rank","num_of_Essays", "num_of_Scores","num_of_Advice","num_of_ECs",
                     "num_of_Sports", "num_of_schools", "SAT", "ACT"]

        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
    
        for user_url in list_of_user_url:
            if user_url in user_dictionary: 
                continue
            else:
                try:
                    user_dictionary[user_url] = None
                    driver = webdriver.Chrome()     
                    driver.get(user_url)

                    time.sleep(2)

                except Exception as e: 
                    print("failed to load user_url")

                try:
                    #username
                    username = driver.find_element_by_xpath("//div[@class='user_name']").text
                    #school
                    current_school = driver.find_element_by_xpath("//div[@class='user_school']").text
                    #major
                    user_major = driver.find_element_by_xpath("//div[@class='user_major']").text
                    hometown, gender, ethnicity, English_FL = None, None, None, None
                    try: 
                        for i in range(len(driver.find_elements_by_xpath("//div[@class='hidethese']/span"))):
                            name = driver.find_elements_by_xpath("//div[@class='hidethese']/span")[i].get_attribute("innerHTML")

                            if name == "Hometown":
                                hometown = driver.find_elements_by_xpath("//div[@class='hidethese']/span")[i+1].get_attribute("innerHTML")

                            elif name == "Gender":
                                gender = driver.find_elements_by_xpath("//div[@class='hidethese']/span")[i+1].get_attribute("innerHTML")
                            elif name == "Ethnicity":
                                ethnicity = driver.find_elements_by_xpath("//div[@class='hidethese']/span")[i+1].get_attribute("innerHTML")

                            elif name == "English First Language":
                                English_FL  = driver.find_elements_by_xpath("//div[@class='hidethese']/span")[i+1].get_attribute("innerHTML")
                    except Exception as e: 
                        print("loop 1 at fault")
                        print(user_url)
                        
                    
                    try: 
                        Admission, class_of, first_gen = None, None, None
                        for i in range(len(driver.find_elements_by_xpath("//div[@class='p_panel_content']/span"))): 
                            name = driver.find_elements_by_xpath("//div[@class='p_panel_content']/span")[i].get_attribute("innerHTML")

                            if name == "Admission":
                                Admission = driver.find_elements_by_xpath("//div[@class='p_panel_content']/span")[i+1].get_attribute("innerHTML")
                            elif name == "Class of":
                                class_of = driver.find_elements_by_xpath("//div[@class='p_panel_content']/span")[i+1].get_attribute("innerHTML")

                            elif name == "First Gen College Student":
                                first_gen = driver.find_elements_by_xpath("//div[@class='p_panel_content']/span")[i+1].get_attribute("innerHTML")
                    except Exception as e: 
                        print("loop 2 at fault")
                        print(user_url)

                    try:
                        #dropdown for schools
                        driver.find_element_by_xpath("//span[@class='tot']").click()

                        #schools accepted to 
                        accepted_schools = driver.find_element_by_xpath("//div[@class='enr acc sub summ']//div[@class='deet']").text
                        
                        #schools waitlisted to
                        waitlisted_schools = driver.find_element_by_xpath("//div[@class='wai sub summ']//div[@class='deet']").text

                        #schools rejected to
                        rejected_schools = driver.find_element_by_xpath("//div[@class='rej sub summ']//div[@class='deet']").text

                        #legacy 
                        legacy = driver.find_element_by_xpath("//span[@class='dataschoollegacy']").text

                        GPA_measure = None
                        try:
                            GPA_measure = driver.find_elements_by_xpath("//div[@class='t_col']/div")[0].get_attribute("innerHTML")
                        except Exception as e: 
                            print("moving on")
#                         #GPA
#                         GPA = driver.find_elements_by_xpath("//div[@class='t_col']/div")[1].get_attribute("innerHTML")

#                         #Rank 
#                         rank = driver.find_elements_by_xpath("//div[@class='t_col']/div")[3]_attribute("innerHTML")
                    except Exception as e: 
                        print("this block at fault")
                        print(user_url)

                    #STATs
                    try:
                        num_of_Essays, num_of_Scores, num_of_Advice, num_of_ECs, num_of_Sports =  None, None, None, None, None
                        for i in range(len(driver.find_elements_by_xpath("//div[@class='stats']/div"))):
                            name = driver.find_elements_by_xpath("//div[@class='stats']/div")[i].text

                            if name == "Essays":
                                num_of_Essays = driver.find_elements_by_xpath("//div[@class='stats']")[i].text

                            elif name == "Scores":
                                num_of_Scores = driver.find_elements_by_xpath("//div[@class='stats']")[i].text

                            elif name == "Advice":
                                num_of_Advice = driver.find_elements_by_xpath("//div[@class='stats']")[i].text

                            elif name == "ECs":
                                num_of_ECs = driver.find_elements_by_xpath("//div[@class='stats']")[i].text

                            elif name == "Sports":
                                num_of_Sports = driver.find_elements_by_xpath("//div[@class='stats']")[i].text

                        num_of_schools = driver.find_elements_by_xpath("//div[@class='stats totalschools']")[0].text
                        
                    except Exception as e:
                        print("loop 3 at fault")
                        print(user_url)


                    SAT = None
                    try: 
                        SAT = driver.find_element_by_xpath("//div[@class='leftscore sc thesat2']").text
                    except Exception as e: 
                        print("no SAT was found")
                    
                    GPA = None
                    try: 
                        GPA = driver.find_element_by_xpath("//div[@class='leftgpa sc']").text
                    except Exception as e: 
                        print("no GPA found!")
                    
                    ACT = None
                    try:
                        ACT = driver.find_element_by_xpath("//div[@class='leftscore2 sc']").text
                    except Exception as e: 
                        print("no ACT was found")


                    user_profile = {

                        "user_url": user_url,
                        "username":username,
                        "current_school":current_school,
                        "user_major":user_major,
                        "hometown":hometown,
                        "gender":gender,
                        "ethnicity":ethnicity,
                        "English_FL":English_FL,
                        "Admission":Admission, 
                        "class_of":class_of,
                        "first_gen":first_gen,
                        "accepted_schools":accepted_schools,
                        "waitlisted_schools":waitlisted_schools,
                        "rejected_schools":rejected_schools,
                        "legacy":legacy,
                        "GPA_measure":GPA_measure,
                         "GPA":GPA,
#                         "rank":rank,
                        "num_of_Essays":num_of_Essays, 
                        "num_of_Scores":num_of_Scores,
                        "num_of_Advice":num_of_Advice,
                        "num_of_ECs":num_of_ECs,
                         "num_of_Sports":num_of_Sports,
                        "num_of_schools":num_of_schools,
                        "SAT":SAT,
                        "ACT":ACT

                    }

                    writer.writerow(user_profile)

                except Exception as e: 
                    print(e)

                driver.close()  
            
            
            

In [43]:
schools = ["Harvard", "Yale", "Penn State", "USC", "Duke", "MIT", "Princeton", "Stanford", "Columbia University", "UPenn", 
          "UCLA", "University of Michigan", "Caltech", "Northwestern", "Chicago", "Brown", "Cornell", "Vanderbilt",
          "UC Berkeley", "Williams", "Dartmouth", "Amherst", "Rice", "Johns Hopkins", "Washington", "Ponoma", "Swarthmore",
          "CMU","Virginia", "Notre Dame", "Bowdoin", "UNC", "University Florida", "Georgetown", "Wellesley","Middlebury",
          "Georgia", "NYU", "Austin", "Emory", "Tuft", "Illinois", "Wisconsin", "Purdue", "Boston", "Arizona",
          "Iowa", "Michigan", "Ohio State", "Maryland", "Alabama", "BYU", "Nebraska", "Minnesota","Illinois", "Indiana", 
          "Rutgers", "Clemson", "University of Virginia", "Wake Forest", "Florida State University", "Boston University",
          "Syracuse", "Miami", "Louisville"]

In [44]:
get_user_info(get_profile_urls(schools))

Going through Harvard
0
Going through Yale
310
Going through Penn State
408
Going through USC
576
Going through Duke
576
Going through MIT
862
Going through Princeton
864
Going through Stanford
1043
Going through Columbia University
1349
Going through UPenn
1611
Going through UCLA
2108
Going through University of Michigan
2638
Going through Caltech
3009
Going through Northwestern
3047
Going through Chicago
3048
Going through Brown
3048
Going through Cornell
3048
Going through Vanderbilt
3422
Going through UC Berkeley
3633
Going through Williams
4189
Going through Dartmouth
4312
Going through Amherst
4507
Going through Rice
4598
Going through Johns Hopkins
4733
Going through Washington
4927
Going through Ponoma
5028
error in searching for...Ponoma
Going through Swarthmore
8953
Going through CMU
9021
Going through Virginia
9211
Going through Notre Dame
9213
Going through Bowdoin
9215
Going through UNC
9260
Going through University Florida
9261
error in searching for...University Florida


no ACT was found
no ACT was found
no SAT was found
name 'Ecception' is not defined
no SAT was found
name 'Ecception' is not defined
name 'Ecception' is not defined
no ACT was found
no ACT was found
no SAT was found
no ACT was found
no ACT was found
name 'Ecception' is not defined
no ACT was found
no SAT was found
no SAT was found
no ACT was found
name 'Ecception' is not defined
no SAT was found
no ACT was found
no ACT was found
no ACT was found
no SAT was found
no SAT was found
no ACT was found
no SAT was found
no ACT was found
no ACT was found
no SAT was found
no SAT was found
no SAT was found
no ACT was found
no ACT was found
name 'Ecception' is not defined
no ACT was found
no ACT was found
no ACT was found
no ACT was found
no ACT was found
no ACT was found
no SAT was found
no ACT was found
no SAT was found
no ACT was found
no SAT was found
no ACT was found
no SAT was found
no ACT was found
no ACT was found
no SAT was found
name 'Ecception' is not defined
name 'Ecception' is not defi

no ACT was found
no ACT was found
no SAT was found
no ACT was found
no ACT was found
no SAT was found
no ACT was found
no ACT was found
no SAT was found
no ACT was found
no ACT was found
no ACT was found
no SAT was found
no ACT was found
no SAT was found
no SAT was found
no SAT was found
no ACT was found
no ACT was found
name 'Ecception' is not defined
no ACT was found
name 'Ecception' is not defined
no ACT was found
name 'Ecception' is not defined
name 'Ecception' is not defined
no SAT was found
name 'Ecception' is not defined
name 'Ecception' is not defined
no ACT was found
no SAT was found
no ACT was found
no SAT was found
name 'Ecception' is not defined
no SAT was found
no ACT was found
no SAT was found
name 'Ecception' is not defined
name 'Ecception' is not defined
no ACT was found
no ACT was found
no ACT was found
no ACT was found
no ACT was found
name 'Ecception' is not defined
name 'Ecception' is not defined
no SAT was found
name 'Ecception' is not defined
no SAT was found
no A

KeyError: 'user_url'